In [1]:
# import libraries
import pandas as pd
import numpy as np

In [2]:
# Load All Data, Combine Col-0 and Col-1 to make DateTime index.
dataset = pd.read_csv('household_power_consumption.txt', sep=';', low_memory=False, infer_datetime_format=True, 
                     parse_dates={'Datetime':[0,1]}, index_col=['Datetime'])

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2075259 entries, 2006-12-16 17:24:00 to 2010-11-26 21:02:00
Data columns (total 7 columns):
Global_active_power      object
Global_reactive_power    object
Voltage                  object
Global_intensity         object
Sub_metering_1           object
Sub_metering_2           object
Sub_metering_3           float64
dtypes: float64(1), object(6)
memory usage: 126.7+ MB


In [4]:
dataset.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
2006-12-16 17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2006-12-16 17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
2006-12-16 17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
2006-12-16 17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [5]:
# mark all missing values from ? to Nan
dataset.replace('?', np.nan, inplace=True)
# make all the dataset values float type
dataset = dataset.astype('float32')

In [6]:
# Define function to replace missing values with the exact observation at same time one day before.
def fill_missing(values):
    one_day = 60 * 24
    for row in range(values.shape[0]):
        for col in range(values.shape[1]):
            if np.isnan(values.iloc[row,col]):
                values.iloc[row,col] = values.iloc[row - one_day, col]

In [7]:
# Replace all nan values
fill_missing(dataset)

In [8]:
# Look for total missing values
dataset.isna().sum()

Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64

In [9]:
# Add a new column by subtracting the sum of three defined sub-metering variables from the total active energy
dataset['Sub_metering_4'] = dataset.iloc[:,0]*1000/60 - (dataset.iloc[:,4] + dataset.iloc[:,5] + dataset.iloc[:,6])
dataset.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Sub_metering_4
Datetime,,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.839996,18.4,0.0,1.0,17.0,52.266670
2006-12-16 17:25:00,5.360,0.436,233.630005,23.0,0.0,1.0,16.0,72.333336
2006-12-16 17:26:00,5.374,0.498,233.289993,23.0,0.0,2.0,17.0,70.566666
2006-12-16 17:27:00,5.388,0.502,233.740005,23.0,0.0,1.0,17.0,71.800003
2006-12-16 17:28:00,3.666,0.528,235.679993,15.8,0.0,1.0,17.0,43.099998


In [10]:
# Save the updated s/s
dataset.to_csv('household_power_consumption_updated.csv')

In [11]:
# Read the updated input file.
dataset = pd.read_csv('household_power_consumption_updated.csv', infer_datetime_format=True, parse_dates=['Datetime'], 
                      index_col=['Datetime'])

# As the given data is in minutes, group data Day wise and find sum of total power consumed.
daily_groups = dataset.resample('D')
daily_data = daily_groups.sum()
daily_data.head(10)
daily_data.to_csv('household_power_consumption_days.csv')